In [9]:
import requests
import json

with open("key.txt", 'r') as f:
    key = f.read()
print(key)

3CA293064B9ECCDC101642C75CCD74DE


In [14]:
def retrieve_recent_match_data(key, start_at=None):
    if start_at is None:
        r = requests.get("https://api.steampowered.com/IDOTA2Match_570/GetMatchHistory/V001/?key=" + key)
        if r.status_code != 200:
            raise Exception("Servers might be down")
        else:
            return json.loads(r.text)
    else:
        r = requests.get("https://api.steampowered.com/IDOTA2Match_570/GetMatchHistory/V001/?start_at_match_id=" + str(start_at) + "&key=" + key)
        if r.status_code != 200:
            raise Exception("Either match id was bad or servers might be down")
        else:
            return json.loads(r.text)
        

In [ ]:
private_id = 4294967295 #players that we can not get info from
unique_id = Set([])
recent = None
while len(unique_id) < 30000:
    matches = 